In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

features = pd.read_csv('data_transform/all.csv')

data = features.fillna(100000000)


remove_column = ['IsTest', 'IsTrain', 'IsValidation', 'Date', 'Point', 'CityName'
                 ,u'MeanForMonthDayByPoints', u'MeanForThroughMonthByPoints'
                ,u'MeanForThroughWeekByPoints'
                ,u'MeanForYearByPoints', u'MeanForYearDayByPoints'
                 ,'MeanForYearDay', 'MeanForThroughWeek'
                ]



dataTrain = data[data.IsTrain == 1]
dataTrain = dataTrain[dataTrain.columns.difference(remove_column)]

X_train = dataTrain[dataTrain.columns.difference(['Count'])]
y_train = dataTrain[['Count']]

dataValidation =  data[data.IsTest == 1]
dataValidation = dataValidation[dataValidation.columns.difference(remove_column)]

X_test = dataValidation[dataValidation.columns.difference(['Count'])]
y_test = dataValidation[['Count']]

In [2]:
X_train.columns

Index([u'MeanForMonth', u'MeanForMonthByPoints', u'MeanForMonthDay',
       u'MeanForThroughMonth', u'MeanForWeekDay', u'MeanForWeekDayByPoints',
       u'MeanForYear', u'Month', u'MonthDay', u'ThroughDay', u'ThroughMonth',
       u'ThroughWeek', u'WeekDay', u'Year', u'YearDay'],
      dtype='object')

In [3]:
dtrain = xgb.DMatrix( X_train, label=y_train)
dtest = xgb.DMatrix( X_test, label=y_test)
evallist  = [(dtest,'eval'), (dtrain,'train')]

param = {'max_depth': 6,
 		'silent': 1,
 		'eta': 0.01,
 		'eval_metric': 'mae',
         'subsample' : 0.9,
         'colsample_bytree':0.5,
         'tree_method':'exact'
         #,'rate_drop':0.01
         ,'objective':'reg:tweedie'
 }

num_round = 16000

bst = xgb.train( param, dtrain, num_round, evallist, verbose_eval=200 )

[0]	eval-mae:117.828	train-mae:103.652
[200]	eval-mae:83.3796	train-mae:70.0092
[400]	eval-mae:33.7328	train-mae:26.1213
[600]	eval-mae:26.627	train-mae:20.4233
[800]	eval-mae:25.7372	train-mae:19.4487
[1000]	eval-mae:25.3977	train-mae:18.845
[1200]	eval-mae:25.1022	train-mae:18.353
[1400]	eval-mae:24.7417	train-mae:17.9121
[1600]	eval-mae:24.4466	train-mae:17.5407
[1800]	eval-mae:24.1371	train-mae:17.1916
[2000]	eval-mae:23.8935	train-mae:16.8965
[2200]	eval-mae:23.597	train-mae:16.611
[2400]	eval-mae:23.3788	train-mae:16.3647
[2600]	eval-mae:23.1428	train-mae:16.1388
[2800]	eval-mae:22.8981	train-mae:15.9374
[3000]	eval-mae:22.6619	train-mae:15.7244
[3200]	eval-mae:22.4834	train-mae:15.5427
[3400]	eval-mae:22.2552	train-mae:15.3565
[3600]	eval-mae:22.0727	train-mae:15.1852
[3800]	eval-mae:21.9304	train-mae:15.0113
[4000]	eval-mae:21.7756	train-mae:14.8518
[4200]	eval-mae:21.6588	train-mae:14.7014
[4400]	eval-mae:21.4968	train-mae:14.5568
[4600]	eval-mae:21.3453	train-mae:14.4077
[480

In [4]:
pred = bst.predict(dtest)

In [5]:
y_test = np.array(y_test.Count)

In [6]:
#MAPE
sum( abs((pred - y_test) / y_test) ) * (100. / pred.shape[0])

19.09470622389934